# Install and import libraries

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statistics

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import transformers
from transformers import BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup

from seqeval.metrics import f1_score, accuracy_score

2023-07-05 09:43:05.004063: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 09:43:05.526808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Check GPU availability

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

# **Preprocessing data**

In [3]:
# Reading data
import json


#open the file, and format correctly
f =open('dataset.jsonl', 'r')
json_object = json.dumps(f.readlines(), indent=4)
f.close()

#save better format into file
p = open('sample.txt', 'w')
for i in json_object:
    p.write(i)
p.close()

#open new file, and save each 
j = open('sample.txt', 'r')
text = json.loads(j.read())
j.close()

#compile all json dicts into a list
info = []
for i in text:
    info.append(json.loads(str(i)))
    

#test it
print(info[0]['annotations'])
print(info[0]['text'])
print(info[0]['annotations'][0]['start_offset'])

[{'label': 6, 'start_offset': 17, 'end_offset': 28, 'user': 1, 'created_at': '2023-06-14T14:36:46.643706Z', 'updated_at': '2023-06-14T14:36:46.643737Z'}, {'label': 2, 'start_offset': 89, 'end_offset': 91, 'user': 1, 'created_at': '2023-06-14T14:37:01.615695Z', 'updated_at': '2023-06-26T13:46:16.108486Z'}, {'label': 6, 'start_offset': 247, 'end_offset': 258, 'user': 1, 'created_at': '2023-06-14T14:37:39.635439Z', 'updated_at': '2023-06-14T14:37:39.635488Z'}, {'label': 3, 'start_offset': 461, 'end_offset': 475, 'user': 1, 'created_at': '2023-06-14T14:38:01.779370Z', 'updated_at': '2023-06-14T14:38:01.779403Z'}, {'label': 1, 'start_offset': 477, 'end_offset': 481, 'user': 1, 'created_at': '2023-06-14T14:38:12.949077Z', 'updated_at': '2023-06-26T14:45:46.411185Z'}, {'label': 2, 'start_offset': 498, 'end_offset': 500, 'user': 1, 'created_at': '2023-06-14T14:39:21.688854Z', 'updated_at': '2023-06-26T13:46:09.687470Z'}, {'label': 6, 'start_offset': 540, 'end_offset': 551, 'user': 1, 'created_

In [4]:
def labeler1(sent, text, annotations):
    #sent will be the already parsed sentence, with everything removed
    #text will be the straight sentence, info[i]['text']
    #annotations will be the list of labels, must be info[i]['annotations']
    
    #monitor where in the text we are
    text_index = 0
    
    #monitor which word we are in
    sent_index = 0
    
    #keep track of which annotation label we are applying
    annotation_index = 0
    
    #will return all labels for each words
    labels = []
  
    while text_index < len(text) and sent_index < len(sent):
        if annotation_index >= len(annotations):
            for i in range(len(labels)-1, len(sent)):
                labels.append(data_tags[0])
                return labels
        
        #check if we are at the correct index
        if text[text_index]== sent[sent_index][0]:
            
            #double check
            if len(sent[sent_index])==1 or text[text_index+1]== sent[sent_index][1]:
                
                #if not true and the current label aplies to text behind the index behind current, adjest label that you are looking at
                if text_index > annotations[annotation_index]['end_offset']:
                    annotation_index +=1
                    
                else:
            
                    #check if the current text label falls in a label range.
                    #if true, apply correct label and move text_index and sent_index
                    if text_index >= annotations[annotation_index]['start_offset'] and text_index <= annotations[annotation_index]['end_offset']:
                        if len(labels)==0:
                            labels.append(data_tags[2*(annotations[annotation_index]['label'])])
                        elif labels[-1] == data_tags[2*(annotations[annotation_index]['label'])] or labels[-1] == data_tags[2*(annotations[annotation_index]['label'])-1]:
                            labels.append(data_tags[2*(annotations[annotation_index]['label'])])
                        else:
                            labels.append(data_tags[2*(annotations[annotation_index]['label'])-1])
                    
                    #if no label applies, apply '0' tag
                    else:
                        labels.append(data_tags[0])
                    
                    #adjust which word you are looking at and text_index
                    text_index += len(sent[sent_index])
                    sent_index += 1
                    
                
            else:
                text_index+=1
        else:
            text_index+=1
            
            
    return labels
        
 

In [5]:
def labeler2(sent, text, annotations):
    #sent will be the already parsed sentence, with everything removed
    #text will be the straight sentence, info[i]['text']
    #annotations will be the list of labels, must be info[i]['annotations']
    
    #monitor where in the text we are
    text_index = 0
    
    #keep track of which annotation label we are applying
    annotation_index = 0
    
    #will return all labels for each words
    labels = []
    
    for i in range(0, len(sent)):
        if len(annotations) ==0:
            return [data_tags[0] for j in range(len(sent[i]))]
                    
        elif text_index == annotations[annotation_index]['start_offset']:
            
            labels.append(data_tags[2 * annotations[annotation_index]['label']-1])
            
        elif text_index >= annotations[annotation_index]['start_offset'] and text_index <= annotations[annotation_index]['end_offset']:
            if labels[i-1]==data_tags[0]:
                
                labels.append(data_tags[2 * annotations[annotation_index]['label']-1])
            else:
               
                labels.append(data_tags[(2 * annotations[annotation_index]['label'] ) ])
            
        elif text_index > annotations[annotation_index]['end_offset']:
            if text_index + 1 == annotations[annotation_index]['start_offset']:
                
                labels.append(data_tags[2 * annotations[annotation_index]['label']-1])
            else: 
                annotation_index+=1
               
                labels.append(data_tags[0])
                if annotation_index>=len(annotations):
                    annotation_index = annotation_index -1
        else:
        
            labels.append(data_tags[0])
            
        text_index+= (len(sent[i])+1)
    return labels
    

In [6]:
def labeler3(sent, text, annotations):
    #sent will be the already parsed sentence, with everything removed
    #text will be the straight sentence, info[i]['text']
    #annotations will be the list of labels, must be info[i]['annotations']
    
    #monitor where in the text we are
    text_index = 0
    
    #monitor which word we are in
    sent_index = 0
    
    #keep track of which annotation label we are applying
    annotation_index = 0
    
    #will return all labels for each words
    labels = []
    
    

In [7]:
# To use the BERT, you must use the BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True)


In [8]:
import re

def splitter1():
    return [re.findall(r"[\w']+", info[i]['text']) for i in range(len(info))]

def splitter2():
    return [info[i]['text'].split() for i in range(len(info))]
    
def splitter3():
    l = []
    for i in range(len(info)):
        sent = tokenizer.tokenize(info[i]['text'])
        for i in range(len(sent)):
            if sent[i][0]=='#':
                sent[i] = sent[i][2:]
        print(sent)       
        l.append(sent)
    return l
        
    
    

In [9]:
def processor(splitmethod, labelmethod):
    s = splitmethod()
    
    l=[]

    for i in range(len(s)):
        l.append(labelmethod(s[i], info[i]['text'], info[i]['annotations'] ))
    
    return s, l

In [10]:
print(tokenizer.tokenize(info[0]['text']))

['the', 'influence', 'of', 'γ', '-', 'radiation', 'on', 'the', 'physical', 'characteristics', 'and', 'sorption', 'properties', 'for', 'am', 'of', 'the', 'solid', 'extract', '##ant', 'axi', '##onit', 'mn', '##d', '40', '##t', '(', 'tv', '##ex', ')', 'synthesized', 'by', 'axi', '##on', '-', 'rare', 'and', 'noble', 'metals', 'js', '##c', '(', 'perm', ')', 'was', 'studied', '.', 'with', 'an', 'increase', 'of', 'the', 'absorbed', 'dose', 'of', 'γ', '-', 'radiation', ',', 'the', 'granul', '##ometric', 'composition', ',', 'bulk', 'density', ',', 'and', 'specific', 'volume', 'of', 'tv', '##ex', 'change', 'insignificant', '##ly', '.', 'the', 'ir', 'spectra', 'of', 'tv', '##ex', 'demonstrate', 'significant', 'radiation', 'degradation', 'at', 'an', 'absorbed', 'radiation', 'dose', 'from', '1', '.', '4', 'to', '2', '.', '5', 'mg', '##y', '.', 'tv', '##ex', 'is', 'an', 'effective', 'am', 'sor', '##ben', '##t', 'when', 'irradiated', 'to', 'an', 'absorbed', 'γ', '-', 'radiation', 'dose', 'of', 'no', 

In [11]:


print(splitter3())

#create labels
data_tags = ['0','C', 'C-cont' , 'M', 'M-cont', 'Q', 'Q-cont', 'T' , 'T-cont', 'A' , 'A-cont', 'R', 'R-cont']

sentences, labels = processor(splitter1, labeler1)



['the', 'influence', 'of', 'γ', '-', 'radiation', 'on', 'the', 'physical', 'characteristics', 'and', 'sorption', 'properties', 'for', 'am', 'of', 'the', 'solid', 'extract', 'ant', 'axi', 'onit', 'mn', 'd', '40', 't', '(', 'tv', 'ex', ')', 'synthesized', 'by', 'axi', 'on', '-', 'rare', 'and', 'noble', 'metals', 'js', 'c', '(', 'perm', ')', 'was', 'studied', '.', 'with', 'an', 'increase', 'of', 'the', 'absorbed', 'dose', 'of', 'γ', '-', 'radiation', ',', 'the', 'granul', 'ometric', 'composition', ',', 'bulk', 'density', ',', 'and', 'specific', 'volume', 'of', 'tv', 'ex', 'change', 'insignificant', 'ly', '.', 'the', 'ir', 'spectra', 'of', 'tv', 'ex', 'demonstrate', 'significant', 'radiation', 'degradation', 'at', 'an', 'absorbed', 'radiation', 'dose', 'from', '1', '.', '4', 'to', '2', '.', '5', 'mg', 'y', '.', 'tv', 'ex', 'is', 'an', 'effective', 'am', 'sor', 'ben', 't', 'when', 'irradiated', 'to', 'an', 'absorbed', 'γ', '-', 'radiation', 'dose', 'of', 'no', 'more', 'than', '0', '.', '6',

['the', 'presented', 'results', 'show', 'that', 'when', 'using', 'the', 'sorption', 'method', 'of', 'purification', ',', 'which', 'includes', 'the', '90', 'y', 'sorption', 'stages', 'on', 'the', 'axi', 'onit', 'mn', 'd', '40', 't', 'sor', 'ben', 't', ',', 'washing', 'and', 'elution', 'of', 'yt', 'tri', 'um', 'with', '0', '.', '01', 'm', 'hcl', ',', 'deep', 'purification', 'of', 'the', 'stripp', 'ed', '90', 'y', 'from', 'residual', 'stro', 'nt', 'ium', 'amounts', 'as', 'well', 'as', 'from', 'equipment', 'corrosion', 'products', ',', 'is', 'observed', '.']
['the', 'purification', 'factors', '(', 'kp', 'ur', ')', 'of', 'yt', 'tri', 'um', 'from', 'stro', 'nt', 'ium', ',', 'chromium', ',', 'manganese', ',', 'nickel', ',', 'and', 'lead', 'are', 'more', 'than', '100', '.', 'the', 'kp', 'ur', 'for', 'iron', 'and', 'zircon', 'ium', 'ions', 'is', '54', 'and', '40', ',', 'respectively', '.', 'the', 'lowest', 'purification', 'efficiency', 'is', 'observed', 'for', 'impurities', 'of', 'aluminum', ',

In [12]:

# Determine the list of tags
tag_values = data_tags
print(tag_values)

tag_values.append("PAD")
print(tag_values)

tag2idx = {t: i for i, t in enumerate(tag_values)}
print(tag2idx)
     

['0', 'C', 'C-cont', 'M', 'M-cont', 'Q', 'Q-cont', 'T', 'T-cont', 'A', 'A-cont', 'R', 'R-cont']
['0', 'C', 'C-cont', 'M', 'M-cont', 'Q', 'Q-cont', 'T', 'T-cont', 'A', 'A-cont', 'R', 'R-cont', 'PAD']
{'0': 0, 'C': 1, 'C-cont': 2, 'M': 3, 'M-cont': 4, 'Q': 5, 'Q-cont': 6, 'T': 7, 'T-cont': 8, 'A': 9, 'A-cont': 10, 'R': 11, 'R-cont': 12, 'PAD': 13}


In [13]:
for i in range(len(labels[99])):
    print(sentences[99][i],labels[99][i])

Experiments 0
were 0
performed 0
with 0
triamylphosphine 0
oxide A
TAPO A-cont
octyl A-cont
phenyl A-cont
N A-cont
N A-cont
diisobutylcarbamoylmethylphosphine A-cont
oxide A-cont
CMPO A-cont
Ekos 0
Russia 0
and 0
Eu C
NO3 C-cont
3 C-cont
6H2O C-cont
Vekton 0


In [14]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels


In [15]:
tokenized_texts_and_labels = [ tokenize_and_preserve_labels(sent, labs) for sent, labs in zip(sentences, labels)]

tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [16]:
# MAX_LEN is the maximum length of a sequence
MAX_LEN = 64 # 64 or 128 or ...
bs = 5 # batch size


In [17]:
# Use Padding to equalize the length of sentences
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")


In [18]:
# Do not mask values that are zero
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [19]:
# Split data to train and validation. %90 for train and %10 for validation
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)

# Each mask contains 10% of a sentence
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

# The input of the BERT should be the tensors produced using PyTorch
# So convert all inputs and labels into torch tensors
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)

tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)

tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

# Create the DataLoader for our training set
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

print(train_sampler)

# Create the DataLoader for our validation set
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)


# **Build model**

In [20]:
# Load BERT Model

model = BertForTokenClassification.from_pretrained(
    "allenai/scibert_scivocab_uncased",
    num_labels=len(tag2idx), # The number of output labels
    output_attentions = False, # Whether the model returns attention weights.
    output_hidden_states = False # Whether the model returns all hidden-states.
)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [21]:
# Tell PyTorch to run this model on the GPU
model.cuda();

In [22]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr= 1e-5, # learning-rate default is 3e-5
    eps=1e-8 # adam-epsilon default is 1e-8
)


/home/ecasey/anaconda3/envs/torch/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
epochs = 100

max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# **Training**

In [24]:
def acc():
    accuracy_list = []
    
    for i in range(len(sentences)):

        tokenized_sentence = tokenizer.encode(sentences[i])
        input_ids = torch.tensor([tokenized_sentence]).cuda()

        with torch.no_grad():
            output = model(input_ids)

        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

        # join bpe split tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(tag_values[label_idx])
                new_tokens.append(token)

        new_labels = new_labels[1:-2]
        accuracy_list.append(accuracy_score(new_labels, labels[i]))

        #print("\nPredict labels: ",new_labels)
        #print("Actual labels: ",labels[i])

    return statistics.mean(accuracy_list)



In [ ]:
## Store the average loss after each epoch so we can plot them.
import copy
loss_values, validation_loss_values = [], []

test_acc = []

max_test = 0

for e in trange(epochs, desc="Epoch"):
    
    
    
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("\nAverage train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("\nValidation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]

    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    #print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()
    a = acc()
    test_acc.append(a)
    if a * .99 >= max_test:
        max_test = a
        torch.save(model, "test.ck")
    

Epoch:   0%|                                                                                                 | 0/100 [00:00<?, ?it/s]


Average train loss: 1.639241762459278

Validation loss: 0.899729331334432
Validation Accuracy: 0.7295208655332303



Epoch:   1%|▉                                                                                        | 1/100 [00:06<10:12,  6.18s/it]


Average train loss: 0.8746523521840572

Validation loss: 0.5155617992083231
Validation Accuracy: 0.8330757341576507



Epoch:   2%|█▊                                                                                       | 2/100 [00:08<06:06,  3.74s/it]

...

Epoch:  62%|██████████████████████████████████████████████████████▌                                 | 62/100 [02:08<01:16,  2.00s/it]


Average train loss: 0.0021415715467204186

Validation loss: 0.2166027973095576
Validation Accuracy: 0.9675425038639877



Epoch:  63%|███████████████████████████████████████████████████████▍                                | 63/100 [02:10<01:14,  2.00s/it]


Average train loss: 0.0020704703565570526

Validation loss: 0.21611667176087698
Validation Accuracy: 0.9659969088098919



Epoch:  64%|████████████████████████████████████████████████████████▎                               | 64/100 [02:12<01:11,  2.00s/it]

In [ ]:
# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_values, 'b-o', label="training loss")
plt.plot(validation_loss_values, 'r-o', label="validation loss")

# Label the plot.
plt.title("Learning curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

# Performance on test data

In [ ]:
model = torch.load("test.ck")
model.eval()

print(acc())

In [ ]:
print(test_acc)

In [ ]:
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(test_acc, 'b-o', label="test accuracy")


# Label the plot.
plt.title("")
plt.xlabel("Epoch")
plt.ylabel("Test Accuracy")
plt.legend()

plt.show()

In [ ]:
acc()


In [ ]:
sentences, labels = processor(splitter2, labeler2)

acc()